# 基于PyTorch架构，使用scikit-learn进行超参搜索

In [ ]:
pip install skorch

#### skorch为PyTorch模型提供与scikit-learn兼容的API
#### skorch中，有分类神经网络的NeuralNetClassifier和回归神经网络的NeuralNetRegressor
#### reference: https://skorch.readthedocs.io/en/latest/

In [34]:
import numpy as np 
import pandas as pd
import warnings

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import GridSearchCV

import skorch
from skorch import NeuralNetClassifier 

In [ ]:
# Read data 
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

# Feature engineering
## Proprecess data
train['Sex'].replace('male', 0 ,inplace= True)
train['Sex'].replace('female', 1 ,inplace= True)
train['Relation'] = train['SibSp']+train['Parch']
train['Ifhavekid'] = train['Parch']
train['Ifhavekid'][train['Ifhavekid']>0] = 1

dfresult = pd.DataFrame()
df = pd.get_dummies(train['Embarked'])
df.columns = ['Embarked_' +str(x) for x in df.columns ]
train = pd.concat([train,df],axis = 1)

train.Age = (train.Age-min(train.Age))/(max(train.Age)-min(train.Age))
train.Fare = (train.Fare-min(train.Fare))/(max(train.Fare)-min(train.Fare))

test['Sex'].replace('male', 0 ,inplace= True)
test['Sex'].replace('female', 1 ,inplace= True)
test['Relation'] = test['SibSp']+test['Parch']
test['Ifhavekid'] = test['Parch']
test['Ifhavekid'][test['Ifhavekid']>0] = 1

dfresult = pd.DataFrame()
df = pd.get_dummies(test['Embarked'])
df.columns = ['Embarked_' +str(x) for x in df.columns ]
test = pd.concat([test,df],axis = 1)

test.Age = (test.Age-min(test.Age))/(max(test.Age)-min(test.Age))
test.Fare = (test.Fare-min(test.Fare))/(max(test.Fare)-min(test.Fare))

X = pd.concat([train['Pclass'],
               train['Sex'], train['Age'], train['SibSp'], train['Parch'],  
               train['Relation'], train['Fare']],axis=1)
for column in X:
    if np.any(X[column].isnull()):
        mean_val = np.mean(X[column])
        X[column].fillna(mean_val, inplace=True)

y = train['Survived']

In [ ]:
# 数据准备
X = np.array(X)
y = np.array(y)
input=torch.FloatTensor(X)
label=torch.LongTensor(y)

# create model
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_output, activation=nn.ReLU, dropout_rate=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_feature, 256, bias=False)
        self.act = activation()
        self.dropout = nn.Dropout(dropout_rate) 
        self.fc2 = nn.Linear(256, 128)  
        self.fc3 = nn.Linear(128, 64)  
        self.fc4 = nn.Linear(64, 16)  
        self.out = nn.Linear(16,  n_output) 

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.dropout(x) 
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.out(x)
        return x

# 定义优化器损失函数
net = Net(n_feature=7, n_output=2, activation=nn.ReLU, dropout_rate=0.2)    # n_feature:输入的特征维度,n_output:输出的类别个数

In [61]:
model = NeuralNetClassifier( 
        module = net,
        criterion = torch.nn.CrossEntropyLoss())

In [ ]:
# 查看sklearn中的有效指标名称
sorted(sklearn.metrics.SCORERS.keys())
# 查看model中的有效params名称
print(model.get_params())

In [ ]:
# define the grid search parameters
param_grid = {'batch_size': [16, 32],
              'max_epochs': [10, 50],
              'lr': [0.01],
              'optimizer': [optim.Adam],
              'module__n_feature': [7],
              'module__n_output': [2],
              'module__activation': [nn.ReLU, nn.ELU],
              'module__dropout_rate': [0.1, 0.2]}
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=1, scoring='accuracy')
grid_result=grid.fit(input, label)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# 训练数据
trainloss = []
trainacc = []
for t in range(700):
    out = net(input)                 # 输入input,输出out
    loss = loss_func(out, label)     # 输出与label对比
    
    trainloss.append(loss.item())
    
    prediction = torch.max(out, 1)[1] # 返回index  0返回原值
    pred_y = prediction.data.numpy()
    target_y = label.data.numpy()
    accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
    trainacc.append(accuracy)
    
    optimizer.zero_grad()   # 梯度清零
    loss.backward()         # 前馈操作
    optimizer.step()        # 使用梯度优化器

In [ ]:
# 得出结果
out = net(input) #out是一个计算矩阵，可以用Fun.softmax(out)转化为概率矩阵
out_prob = F.softmax(out)
print(type(out_prob))
print(out_prob)
prediction = torch.max(out, 1)[1] # 返回index  0返回原值
pred_y = prediction.data.numpy()
target_y = label.data.numpy()

# 衡量准确率
accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
print("预测准确率",accuracy)